In [1]:
import pandas as pd 
import numpy as np 

In [2]:
# Load person and household records from standard survey records
person = pd.read_excel(r'J:\Projects\Surveys\HHTravel\Survey2014\Data\Final database\Release 4\2014-pr3-M-hhsurvey-persons.xlsx',
                  sheetname='Data1')
hh = pd.read_excel(r'J:\Projects\Surveys\HHTravel\Survey2014\Data\Final database\Release 4\2014-pr3-M-hhsurvey-households.xlsx',
                  sheetname='Data')

In [3]:
# GPS Weighted trips (in Daysim format)
trip = pd.read_csv(r'\\file2\datateam\Projects\Surveys\HHTravel\Survey2014\Data\Final database\Release 4\Adjusted\trips_2014_adjusted_wt_daysim.csv')

# Non GPS weighted
# trip = pd.read_csv(r'R:\SoundCast\estimation\2014\Surveys\2014 Survey\P5\tripP5.dat', delim_whitespace=True)

In [4]:
trip.columns

Index([u'hhno', u'pno', u'day', u'tour', u'half', u'tseg', u'tsvid', u'opurp',
       u'dpurp', u'oadtyp', u'dadtyp', u'opcl', u'otaz', u'dpcl', u'dtaz',
       u'mode', u'pathtype', u'dorp', u'deptm', u'arrtm', u'endacttm',
       u'trexpfac', u'id', u'travcost', u'travdist', u'travtime', u'tripID',
       u'4k_purp'],
      dtype='object')

In [5]:
# Separate college student trips from regional survey trips
college_trips = trip[trip['hhno'] < 14000000]
trip = trip[trip['hhno'] >= 14000000]

In [6]:
len(trip)

46080

# Vehicle Trip Distribution

In [7]:
# Reclassify household columns for cross-classification

# Number of Workers 0 -> 3+
hh['numworkers_crossclass'] = hh['numworkers']
hh.ix[hh['numworkers'] >= 3, 'numworkers_crossclass'] = '3+'
hh['numworkers_crossclass'] = hh['numworkers_crossclass'].astype('str')

# Household size
hh['hhsize_crossclass'] = hh['hhsize']
hh.ix[hh['hhsize'] >= 4, 'hhsize_crossclass'] = '4+'
hh['hhsize_crossclass'] = hh['hhsize_crossclass'].astype('str')

# Household income
# Note that exact ranges from 2014 do not match 2006
# old ranges: 30, 60, 90+
# new ranges: 35, 75, 100+
hh['income_crossclass'] = hh['hh_income_detailed_imp']
hh.ix[hh['hh_income_detailed_imp'] <= 3, 'income_crossclass'] = '1'    # $35k  
hh.ix[(hh['hh_income_detailed_imp'] > 3) & (hh['hh_income_detailed_imp'] <= 5),    # $35-75k
           'income_crossclass'] = '2' 
hh.ix[(hh['hh_income_detailed_imp'] > 5) & (hh['hh_income_detailed_imp'] <= 6),    # $75-100k
           'income_crossclass'] = '3' 
hh.ix[hh['hh_income_detailed_imp'] >= 7, 'income_crossclass'] = '4'    # >$100k

In [8]:
# Reclassify trip purpose for 4k purposes
# Use Daysim User's Guide for data dictionary for trip records

# Home-Based Work (HBW) Trips, directly from home-to-work and work-to-home
trip.ix[(trip['opurp'] == 0) & (trip['dpurp'] == 1), '4k_purp'] = 'HBW'
trip.ix[(trip['opurp'] == 1) & (trip['dpurp'] == 0), '4k_purp'] = 'HBW'

# Home-Based shopping
trip.ix[(trip['opurp'] == 0) & (trip['dpurp'] == 5), '4k_purp'] = 'Home-Based Shopping'
trip.ix[(trip['opurp'] == 5) & (trip['dpurp'] == 0), '4k_purp'] = 'Home-Based Shopping'

# Home-Based Other
trip.ix[(trip['opurp'] == 0) & (trip['dpurp'].isin([3,4,6,7,8,9,10])), '4k_purp'] = 'Home-Based Other'
trip.ix[(trip['opurp'].isin([3,4,6,7,8,9,10]) & (trip['dpurp'] == 0)), '4k_purp'] = 'Home-Based Other'

# NHB Work-to-Other
trip.ix[(trip['opurp'] == 1) & (trip['dpurp'] != 0) & (trip['4k_purp'].isnull()), '4k_purp'] = 'NHB WtO'
trip.ix[(trip['opurp'] != 0) & (trip['dpurp'] == 1) & (trip['4k_purp'].isnull()), '4k_purp'] = 'NHB WtO'

# # NHB Other-to-Other (Destination and Origins are neither work nor home)
trip.ix[(trip['opurp'] != 1) & (trip['dpurp'] != 0) & (trip['opurp'] != 0) 
        & (trip['dpurp'] != 1) & (trip['4k_purp'].isnull()), '4k_purp'] = 'NHB OtO'



In [9]:
np.unique(trip['4k_purp'])

array(['HBW', 'Home-Based Other', 'Home-Based Shopping', 'NHB OtO',
       'NHB WtO', 'School'], dtype=object)

In [10]:
# Merge HH record info to trip records to create cross-class tables
trip_hh = pd.merge(trip, hh, left_on='hhno', right_on='hhid', how='left')

In [11]:
# total in average
weights_hbw = trip_hh['trexpfac'].sum()
my_trip = trip_hh
my_trip['travdist_fac'] = my_trip['travdist']*my_trip['trexpfac']
my_trip['travtime_fac'] = my_trip['travtime']*my_trip['trexpfac']
length_average = my_trip['travdist_fac'].sum() / weights_hbw
time_average = my_trip['travtime_fac'].sum() / weights_hbw

In [12]:
print length_average
print time_average

5.67894243327
15.8926747061


In [13]:
def trip_distribution(my_trip, purp, income, my_measure, my_measure_fac, my_measure_name):
    #my_trip = my_trip[my_trip['4k_purp'] == purp]
    # if it is HBW, we want to break it down to 4 income groups; for other purposes, no need to create income diversity.
    print purp, 'index: ', my_measure_name
    print 'income group', income
    
    if income not in ['0']:
        weights_hbw = my_trip[(my_trip['4k_purp'] == purp) & (my_trip['income_crossclass'] == income)].sum()[['trexpfac']]
        my_trip[my_measure_fac] = my_trip[my_measure]*my_trip['trexpfac']
        trip_purp = my_trip[(trip_hh['4k_purp'] == purp)& (my_trip['income_crossclass'] == income)].sum()[[my_measure_fac]]
        
    else: 
        weights_hbw = my_trip[trip_hh['4k_purp'] == purp].sum()[['trexpfac']]
        my_trip[my_measure_fac] = my_trip[my_measure]*my_trip['trexpfac']
        trip_purp = my_trip[my_trip['4k_purp'] == purp].sum()[[my_measure_fac]]
    
    measure_output = trip_purp[my_measure_fac]/weights_hbw['trexpfac']
    print measure_output 


In [14]:
for purp in ['Home-Based Other', 'Home-Based Shopping', 'NHB OtO', 'NHB WtO', 'HBW']:
    trip_distribution(trip_hh, purp, '0', 'travdist', 'travdist_fac', 'Length')

Home-Based Other index:  Length
income group 0
4.67509704279
Home-Based Shopping index:  Length
income group 0
3.88243292254
NHB OtO index:  Length
income group 0
4.22727345976
NHB WtO index:  Length
income group 0
5.9871505205
HBW index:  Length
income group 0
10.8541905913


In [15]:
for i in ['1', '2', '3', '4']:
    trip_distribution(trip_hh, 'HBW', i, 'travdist', 'travdist_fac', 'Length')

HBW index:  Length
income group 1
8.63097525979
HBW index:  Length
income group 2
10.4166773793
HBW index:  Length
income group 3
12.0579781958
HBW index:  Length
income group 4
11.6090010026


In [16]:
for purp in ['Home-Based Other', 'Home-Based Shopping', 'NHB OtO', 'NHB WtO', 'HBW']:
    trip_distribution( trip_hh, purp, '0', 'travtime', 'travtime_fac', 'Time')

Home-Based Other index:  Time
income group 0
14.7495868866
Home-Based Shopping index:  Time
income group 0
12.3391788421
NHB OtO index:  Time
income group 0
12.7242177988
NHB WtO index:  Time
income group 0
15.9411214967
HBW index:  Time
income group 0
24.8816881379


In [17]:
for i in ['1', '2', '3', '4']:
    trip_distribution(trip_hh, 'HBW', i, 'travtime', 'travtime_fac', 'Time')

HBW index:  Time
income group 1
21.3736774788
HBW index:  Time
income group 2
24.197246553
HBW index:  Time
income group 3
26.3553786343
HBW index:  Time
income group 4
26.2421749531


# Vehicle Trip Length Frequency

In [18]:
# trip length 
trip_hh['length_crossclass'] = trip_hh['travdist']
trip_hh.ix[trip_hh['travdist'] <= 0, 'length_crossclass'] = '0'

for i in range(25): 
    j = i + 1
    trip_hh.ix[(trip_hh['travdist'] > i) & (trip_hh['travdist'] <= j), 'length_crossclass'] = str(i)
    
trip_hh.ix[trip_hh['travdist'] >= 25, 'length_crossclass'] = '25'  

In [19]:
def trip_length_frequency(my_trip, purp, income, my_measure_name):
    print purp, 'index: ', my_measure_name
    print 'income group', income
    
    if income not in ['0']: #specified income group
        trip_purp = my_trip[(my_trip['4k_purp'] == purp) & (my_trip['income_crossclass'] == income)].groupby(['length_crossclass']).sum()[['trexpfac']].astype(int)
    else:
        trip_purp = my_trip[my_trip['4k_purp'] == purp].groupby(['length_crossclass']).sum()[['trexpfac']].astype(int)
        
    total = trip_purp['trexpfac'].sum()
    print total
    trip_purp['share'] = trip_purp['trexpfac'] / total
    length_output = pd.DataFrame(trip_purp[['trexpfac', 'share']])
    length_output.columns = [['Total Trips', my_measure_name]]
    print length_output

In [20]:
for purp in ['Home-Based Other', 'Home-Based Shopping', 'NHB OtO', 'NHB WtO', 'HBW']:
    trip_length_frequency(trip_hh, purp, '0', 'Share of Trips')

Home-Based Other index:  Share of Trips
income group 0
4638126
                   Total Trips  Share of Trips
length_crossclass                             
0                      1134478        0.244598
1                       879621        0.189650
10                       84108        0.018134
11                       79546        0.017150
12                       66877        0.014419
13                       35583        0.007672
14                       42733        0.009213
15                       33967        0.007323
16                       28869        0.006224
17                       30398        0.006554
18                       13512        0.002913
19                       16890        0.003642
2                       562152        0.121202
20                       22042        0.004752
21                       12694        0.002737
22                       20882        0.004502
23                       23307        0.005025
24                       11042        0.0023

In [21]:
for income in ['1', '2', '3', '4']:
    trip_length_frequency(trip_hh, 'HBW', income, 'Share of Trips')

HBW index:  Share of Trips
income group 1
360269
                   Total Trips  Share of Trips
length_crossclass                             
0                        45243        0.125581
1                        43262        0.120082
10                       19714        0.054720
11                       23464        0.065129
12                       10423        0.028931
13                        9036        0.025081
14                        4999        0.013876
15                        5563        0.015441
16                        4738        0.013151
17                        4385        0.012171
18                        3334        0.009254
19                        4748        0.013179
2                        18754        0.052056
20                        5309        0.014736
21                        5585        0.015502
22                        4421        0.012271
23                         350        0.000971
24                        1047        0.002906
25         

# Mode Choice 

In [22]:
#Total daily
my_trip_mode = pd.DataFrame(trip_hh.groupby(['mode']).sum()['trexpfac'].astype(int))
total = my_trip_mode['trexpfac'].sum().astype(int)
my_trip_mode['mode_share'] = my_trip_mode['trexpfac'] / total
my_trip_mode.columns = [['Total Trips', 'Share of Mode']]

In [23]:
total

13521309

In [24]:
my_trip_mode

,Total Trips,Share of Mode
mode,,
1,1492960,0.110415
2,176345,0.013042
3,5843461,0.432167
4,2894030,0.214035
5,2026432,0.149870
6,665198,0.049196
8,350864,0.025949
9,72019,0.005326


In [26]:
def trip_mode_distribution(purp, income, my_trip):
    if income not in['0']: 
        trip_purp = my_trip[(my_trip['4k_purp'] == purp)&(my_trip['income_crossclass'] == income)].groupby(['mode']).sum()[['trexpfac']].astype(int)
        weight = trip_purp['trexpfac'].sum().astype(int)
        trip_purp['share'] = trip_purp['trexpfac'] / weight
        print 
       
    else: 
        if purp in ['NHB OtO', 'NHB WtO']: 
            trip_purp = my_trip[(my_trip['4k_purp'] == 'NHB WtO') | (my_trip['4k_purp'] == 'NHB OtO')].groupby(['mode']).sum()[['trexpfac']].astype(int)
            weight = trip_purp['trexpfac'].sum().astype(int)
            trip_purp['share'] = trip_purp['trexpfac'] / weight
        
        else: 
            trip_purp = my_trip[(my_trip['4k_purp'] == purp)].groupby(['mode']).sum()[['trexpfac']].astype(int)
            weight = trip_purp['trexpfac'].sum().astype(int)
            trip_purp['share'] = trip_purp['trexpfac'] / weight
    print 'total trips are: ', weight
    print trip_purp
    measure_outcome = pd.DataFrame(trip_purp[['trexpfac', 'share']])
    measure_outcome.columns = [['Total Trips', 'Mode Share of Trip']]
    
    print measure_outcome 


In [27]:
trip_mode_distribution('Home-Based Other', '0', trip_hh)

total trips are:  4638134
      trexpfac     share
mode                    
1       738475  0.159218
2        45336  0.009775
3      1535230  0.331002
4      1244942  0.268414
5       948082  0.204410
6       102368  0.022071
8         6886  0.001485
9        16815  0.003625
      Total Trips  Mode Share of Trip
mode                                 
1          738475            0.159218
2           45336            0.009775
3         1535230            0.331002
4         1244942            0.268414
5          948082            0.204410
6          102368            0.022071
8            6886            0.001485
9           16815            0.003625


In [29]:
trip_mode_distribution('NHB OtO', '0', trip_hh) #NHB OtO and WtO are combined already

total trips are:  3847147
      trexpfac     share
mode                    
1       455531  0.118407
2        39621  0.010299
3      1765334  0.458868
4       864048  0.224594
5       481842  0.125247
6       187436  0.048721
8        35244  0.009161
9        18091  0.004702
      Total Trips  Mode Share of Trip
mode                                 
1          455531            0.118407
2           39621            0.010299
3         1765334            0.458868
4          864048            0.224594
5          481842            0.125247
6          187436            0.048721
8           35244            0.009161
9           18091            0.004702


In [30]:
trip_mode_distribution('HBW', '1', trip_hh)


total trips are:  360282
      trexpfac     share
mode                    
1        20070  0.055706
2         8674  0.024076
3       238874  0.663020
4        17149  0.047599
5        12353  0.034287
6        61400  0.170422
9         1762  0.004891
      Total Trips  Mode Share of Trip
mode                                 
1           20070            0.055706
2            8674            0.024076
3          238874            0.663020
4           17149            0.047599
5           12353            0.034287
6           61400            0.170422
9            1762            0.004891


In [32]:
trip_mode_distribution('HBW', '2', trip_hh)


total trips are:  371187
      trexpfac     share
mode                    
1         8875  0.023910
2         7262  0.019564
3       297181  0.800623
4        18682  0.050330
5         7135  0.019222
6        30585  0.082398
9         1467  0.003952
      Total Trips  Mode Share of Trip
mode                                 
1            8875            0.023910
2            7262            0.019564
3          297181            0.800623
4           18682            0.050330
5            7135            0.019222
6           30585            0.082398
9            1467            0.003952


In [33]:
trip_mode_distribution('HBW', '3', trip_hh)


total trips are:  371187
      trexpfac     share
mode                    
1         8875  0.023910
2         7262  0.019564
3       297181  0.800623
4        18682  0.050330
5         7135  0.019222
6        30585  0.082398
9         1467  0.003952
      Total Trips  Mode Share of Trip
mode                                 
1            8875            0.023910
2            7262            0.019564
3          297181            0.800623
4           18682            0.050330
5            7135            0.019222
6           30585            0.082398
9            1467            0.003952


In [34]:
trip_mode_distribution('HBW', '4', trip_hh)


total trips are:  906247
      trexpfac     share
mode                    
1        17923  0.019777
2        31982  0.035291
3       659330  0.727539
4        69593  0.076793
5        24167  0.026667
6        97107  0.107153
9         6145  0.006781
      Total Trips  Mode Share of Trip
mode                                 
1           17923            0.019777
2           31982            0.035291
3          659330            0.727539
4           69593            0.076793
5           24167            0.026667
6           97107            0.107153
9            6145            0.006781


# Time of the Day

In [35]:
trip_hh['deptm'].max()

1439

In [36]:
trip_hh['time'] = trip_hh['deptm']
trip_hh.ix[(trip_hh['deptm'] >= 360)&(trip_hh['deptm'] < 540), 'time'] = 'AM'
trip_hh.ix[(trip_hh['deptm'] >= 540)&(trip_hh['deptm'] < 900), 'time'] = 'MD'
trip_hh.ix[(trip_hh['deptm'] >= 900)&(trip_hh['deptm'] < 1080), 'time'] = 'PM'
trip_hh.ix[(trip_hh['deptm'] >= 1080)&(trip_hh['deptm'] < 1320), 'time'] = 'EV'
trip_hh.ix[(trip_hh['deptm'] >= 1320)|(trip_hh['deptm'] < 360), 'time'] = 'NI'

In [37]:
trip_hh['time'].head()

0    NI
1    AM
2    PM
3    PM
4    MD
Name: time, dtype: object

In [38]:
#SOV (mode==3) HBW #1, #2, #3, #4
for income in ['1', '2', '3', '4']:
    print 'income:', income
    print trip_hh[(trip_hh['4k_purp'] == 'HBW') & (trip_hh['mode'] == 3) & (my_trip['income_crossclass'] == income)].groupby(['time']).sum()[['trexpfac']].astype(int)

income: 1
      trexpfac
time          
AM       64888
EV       31645
MD       65851
NI       21850
PM       54638
income: 2
      trexpfac
time          
AM      188239
EV       51209
MD      111918
NI       62844
PM      148437
income: 3
      trexpfac
time          
AM      105047
EV       30534
MD       54362
NI       34319
PM       72916
income: 4
      trexpfac
time          
AM      249706
EV       62009
MD      105439
NI       62263
PM      179912


In [24]:
np.unique(trip_hh['4k_purp'])[-7:]

array([nan, nan, 'HBW', 'Home-Based Other', 'Home-Based Shopping',
       'NHB OtO', 'NHB WtO'], dtype=object)

In [47]:
trip_hh[((trip_hh['4k_purp'] == 'Home-Based Other') | (trip_hh['4k_purp'] == 'Home-Based Shopping')  | (trip_hh['4k_purp'] == 'NHB OtO') ) & (trip_hh['mode'] == 3)].groupby(['time']).sum()[['trexpfac']].astype(int)

,trexpfac
time,
AM,303839
EV,629672
MD,1245262
NI,104741
PM,718420


In [48]:
trip_hh[(trip_hh['mode'] == 4)].groupby(['time']).sum()[['trexpfac']].astype(int)

,trexpfac
time,
AM,472883
EV,595562
MD,1013732
NI,68400
PM,743452


In [49]:
trip_hh[(trip_hh['mode'] == 5)].groupby(['time']).sum()[['trexpfac']].astype(int)

,trexpfac
time,
AM,377389
EV,411380
MD,567448
NI,31857
PM,638356


In [30]:
NHB_df = pd.DataFrame()
print NHB_df
for purp in ['NHB OtO', 'NHB WtO']:
    print 'purpose:', purp
    NHB_df = pd.DataFrame(trip_hh[(trip_hh['4k_purp'] == purp) ].groupby(['time']).sum()[['trexpfac']].astype(int))
    print NHB_df

Empty DataFrame
Columns: []
Index: []
purpose: NHB OtO
      trexpfac
time          
AM      238429
EV      488948
MD     1577199
NI       29322
PM      868144
purpose: NHB WtO
      trexpfac
time          
AM      246582
EV      116068
MD      844504
NI       32474
PM      553157
